<a href="https://colab.research.google.com/github/hlydecker/are-you-wearing-a-mask/blob/main/cv_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision Demo

## Introduction

This notebook shows how you can use a Google Colab VM for training YOLO object detection models using data from Kaggle.

**Links**
- [Face Mask Detection](https://github.com/hlydecker/are-you-wearing-a-mask) GitHub repository.
- [Face Masks Dataset](https://www.kaggle.com/datasets/henrylydecker/face-masks) on Kaggle.
- [Prediction Web App](https://huggingface.co/spaces/hlydecker/are-you-wearing-a-mask) on Huggingface Spaces.

# Setup

Before we get started, we need to set a few things up. To train our model, we will be using a Google Colab notebook. If you aren't familiar with Colab, it is a service provided by Google where users can run notebooks on Google Cloud Platform virtual machines. At the moment, they provide access to GPUs for users on the free tier. We will be using a Google Colab GPU machine for this demonstation.

## Step 1: Install core dependencies

We need to clone the YOLOv5 GitHub repository, the mask detection GitHub repository, and install kaggle. 
We also need to make a .kaggle directory.

In [ ]:
!git clone https://github.com/ultralytics/yolov5 
!git clone https://github.com/hlydecker/are-you-wearing-a-mask

%pip install kaggle
!mkdir ~/.kaggle

## Step 2: Download data from Kaggle

To intereact with Kaggle datasets, you need to have a user account. 
Once you've created a user account, download your `kaggle.json` API key. 
You'll then need to upload your API key into the Colab's content directory (where we are working right now).

After the upload has finished, we will set things up so that Kaggle knows where our API key is.
Then we can simply download and unzip our dataset.

In [ ]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d henrylydecker/face-masks
!unzip face-masks.zip

## Step 3: Install YOLOv5

We are now ready to install YOLOv5. 

If this all works correctly, we can then print out a message which will let us know if everything is set up right. 
Lookout to make sure that this message has "CUDA" in it.

In [ ]:
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow wandb

import torch
import os
from IPython.display import Image, clear_output  # to display images

In [ ]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

## Step 4: Training

Before we can train, we also want to make sure that we have our dataset's .yaml file set correctly, and that everything is in the correct place. 
You might need to move files around a bit!
HINT: Colab has a super useful feature allowing you to copy paths to items using the file explorer.

To train our YOLO model, we need to run the `train.py` script, with a few parameters set.

These parameters are:
- `img`: this tells the model what size of image to work with. YOLOv5 is built for 640p images, so we will set this to 640.
- `batch`: this defines the batch size for training. This should be set to the largest batch size allowed by our GPU's VRAM. I've set this to 16 for our Google Colab GPU VM.
- `epochs`: this tells the model how many training/evaluation rounds to run. Ideally we should set this value to at least 300. The model will stop training if things don't improve for 100 epochs, but generally 300 is a great starting point.
- `data`: this points YOLO to our dataset's .yaml file. This .yaml file includes info about where the data is, and what classes are in the dataset.
- `project`: a name for our project. Defaults to something generic, so feel free to pick a more informative name!
- `weights`: defines the pre trained starting point for our model to fine tune from. I've used YOLOv5s because it is accurate enough while being lean enough to run on limited compute hardware. Fancier, bigger, better models are more complicated and need more compute.

Tip: `train.py` has many more arguments! Check them out to learn more about all of the ways you can control and modify your model training.

In [ ]:
%cd ..

In [ ]:
!python yolov5/train.py --img 640 --batch 16 --epochs 100 --data data.yaml --project cv-demo --weights yolov5s.pt

## Step 5: Prediction

Once we have a trained model, we can experiment with predictions!

Tell the model what to predict on using the `--source` parameter. 

- If you put a directory here, the model will predict on any images inside the directory.
- If you paste a youtube link the model will predict on the youtube video.
- 0 for webcam prediction. You need to do fancy stuff with javascript to make this work on Colab.

Predictions are saved in the `yolov5/runs/detect` directory.

In [ ]:
!python yolov5/detect.py --source "https://www.youtube.com/watch?v=ayy6Wb8zcyc" --weights "/content/cv-demo/exp/weights/best.pt"  